In [169]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from subprocess import check_output

In [170]:
# # I use default email library just for simplicity. 
# # We extract email artificials and content from raw text

# def email_from_string(raw_email):
#     msg = email.message_from_string(raw_email)
    
#     content = []
#     for part in msg.walk():
#         if part.get_content_type() == 'text/plain':
#             content.append(part.get_payload().lower())
            
#     result = {}
#     for key in msg.keys(): 
#         result[key] = msg[key]
#     result["content"] = ''.join(content)
    
#     return result

In [171]:
# Import the email modules we'll need
import glob
import email
import mailparser
from email import policy
from email.parser import BytesParser

path = '../datawe/raw/Email_Classification/*'
email_types = glob.glob(path)
appendFilesData = []
file_raw_data = [] 
for folder in email_types:
    files = glob.glob(folder+"/*.txt")
    email_type = folder.split('\\')[1]
    for name in files:
        try:
            with open(name) as fp:
                raw_data = fp.read()
                file_raw_data.append(raw_data)
                msg = mailparser.parse_from_string(raw_data)
#                 msg = email_from_string(raw_data)
                #appendFilesData.append(fp.read())
                #msg = email.message_from_string(fp)
                #msg = BytesParser(policy=policy.default).parse(fp)
                #richest = ''.join(msg.get_body(preferencelist=('plain', 'html')).get_content().splitlines(keepends=True)[:])
                appendFilesData.append({
                    "to":msg.to,
                    "from":msg.from_,
                    "subject":msg.subject,
                    "date":msg.date,
#                     "sent":msg["Sent"],
#                     "importance":msg["Importance"],
                    "content":msg.body,
                    "class_to_exec":email_type,
                })
         
        except IOError as exc:
            print('Exception')

In [172]:
data = pd.DataFrame(appendFilesData)
#data['length'] = data['content'].apply(len)
data.head()

,class_to_exec,content,date,from,subject,to
0,Booking,"Dear Rohit,\n \nAs per our conversation a whil...",None,"[(, Meemendra), (, ), (, meemendra.kumar@andbe...",Mr. Nagdev X 04,"[(, Gateway)]"
1,Booking,"Hi Agnes,\n \nI have not received a response t...",None,"[(, Kurien), (, Vinay), (, ), (, vkurien@jones...",RE: Rate Proposal - Taj Connemara,"[(Reservations, Reservations@tajhotels.com)]"
2,Booking,"As per corporate holiday plan, kindly arrange ...",None,"[(, tehp@tatasteel.com[mailto:tehp@tatasteel.c...",TEHP/18/12809: Booking request of Arun Gadamsh...,"[(Reservations Coorg, reservations.coorg@tajho..."
3,Booking,"Dear Gampa ji,\n \nGreetings of the day !\n \n...",None,"[(, INDIAN), (, ), (, indianjourney@gmail.com)]",Valley Period/3 double room/Lerner group,"[(, Neeraj)]"
4,Cancellation,Hi\n\nPlease cancel my booking .,None,"[(, rajivbhatia@ase.ae[mailto:rajivbhatia@ase....",Re: Taj Chandigarh Payment link for reservatio...,"[(Reservations, Reservations@tajhotels.com)]"


In [173]:
data["class_to_exec"].value_counts()

Enquiry         10
Payment          7
Booking          4
Cancellation     3
Name: class_to_exec, dtype: int64

### removing punctuatins and numbers

In [174]:
# def pre_process(text):
#     text = text.translate(str.maketrans('', '', string.punctuation))
#     text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
#     text = [word for word in text if word.isalnum()]
#     words = ""
#     for i in text:
#             stemmer = SnowballStemmer("english")
#             words += (stemmer.stem(i))+" "
#     return words

### Cleaning(punctuations, alpha charachters) and lemantizing 

In [175]:
def clean(doc):
    words_to_exclude = set(stopwords.words('english'))
    exclude = set(string.punctuation)

    word_free = " ".join([i for i in doc.lower().split() if i not in words_to_exclude])
    punc_free = ''.join(ch for ch in word_free if ch not in exclude)
    alpha_free = " ".join(word for word in punc_free.split() if word.isalpha())
    stemmer = SnowballStemmer("english").stem
    stem_free = " ".join(stemmer(stem) for stem in alpha_free.split())
    lemma = WordNetLemmatizer()
    normalized = " ".join(lemma.lemmatize(word) for word in alpha_free.split())

    return normalized

# def clean_data(data):
#     return [clean(doc).split(' ') for doc in data]

In [176]:
# textFeatures[19]

In [177]:
textFeatures = data['content'].copy()
textFeatures = textFeatures.apply(clean)
vectorizer = TfidfVectorizer("english", smooth_idf=False, sublinear_tf=False, analyzer='word')
features = vectorizer.fit_transform(textFeatures)

features_train, features_test, labels_train, labels_test = train_test_split(features, data['class_to_exec'], test_size=0.2, random_state=111)

In [178]:
len(vectorizer.get_feature_names())
#features.get_feature_names()

284

In [179]:
data.head()

,class_to_exec,content,date,from,subject,to
0,Booking,"Dear Rohit,\n \nAs per our conversation a whil...",None,"[(, Meemendra), (, ), (, meemendra.kumar@andbe...",Mr. Nagdev X 04,"[(, Gateway)]"
1,Booking,"Hi Agnes,\n \nI have not received a response t...",None,"[(, Kurien), (, Vinay), (, ), (, vkurien@jones...",RE: Rate Proposal - Taj Connemara,"[(Reservations, Reservations@tajhotels.com)]"
2,Booking,"As per corporate holiday plan, kindly arrange ...",None,"[(, tehp@tatasteel.com[mailto:tehp@tatasteel.c...",TEHP/18/12809: Booking request of Arun Gadamsh...,"[(Reservations Coorg, reservations.coorg@tajho..."
3,Booking,"Dear Gampa ji,\n \nGreetings of the day !\n \n...",None,"[(, INDIAN), (, ), (, indianjourney@gmail.com)]",Valley Period/3 double room/Lerner group,"[(, Neeraj)]"
4,Cancellation,Hi\n\nPlease cancel my booking .,None,"[(, rajivbhatia@ase.ae[mailto:rajivbhatia@ase....",Re: Taj Chandigarh Payment link for reservatio...,"[(Reservations, Reservations@tajhotels.com)]"


In [180]:
# docs = [["hello", "world", "hello"], ["goodbye", "cruel", "world"]]
# indptr = [0]
# indices = []
# data = []
# vocabulary = {}
# for d in docs:
#     for term in d:
#         index = vocabulary.setdefault(term, len(vocabulary))
#         indices.append(index)
#         data.append(1)
# indptr.append(len(indices))
# csr_matrix((data, indices, indptr), dtype=int).toarray()


In [181]:
from scipy.sparse import csr_matrix

In [182]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

svc = SVC(kernel='sigmoid', gamma=1.0)
svc.fit(features_train, labels_train)
prediction = svc.predict(features_test)
accuracy_score(labels_test,prediction)

0.6

In [183]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB(alpha=0.2)
mnb.fit(features_train, labels_train)
prediction = mnb.predict(features_test)
accuracy_score(labels_test,prediction)

0.6

In [184]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(features_train, labels_train)
prediction = lr.predict(features_test)
accuracy_score(labels_test,prediction)

c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6

In [185]:
# vectorizer.vocabulary_

### testing for the runtime sample example

In [186]:
sampleExample = """Hi

Please cancel my booking ."""

In [187]:
# sampleExample

In [188]:
textFeaturesTest = pd.Series([sampleExample])
textFeaturesTest = textFeaturesTest.apply(clean)
featuresTest = vectorizer.transform(textFeaturesTest)


In [189]:
mnb.predict(featuresTest)[0]

'Cancellation'

### writting data into processed csv files

In [190]:
cols = vectorizer.get_feature_names()
dataCsv = features.toarray()
df_to_csv = pd.DataFrame(dataCsv, columns=cols)
df_to_csv = df_to_csv.join(data["class_to_exec"], how="outer")

df_to_csv.to_csv('..\datawe\processed\data.csv')

In [191]:
vectorizer.vocabulary_

{'dear': 84,
 'rohit': 223,
 'per': 193,
 'conversation': 73,
 'ago': 10,
 'please': 198,
 'book': 46,
 'confirm': 68,
 'executive': 101,
 'room': 224,
 'taj': 256,
 'gateway': 114,
 'ganges': 113,
 'december': 86,
 'two': 269,
 'night': 179,
 'hi': 127,
 'agnes': 9,
 'received': 210,
 'response': 221,
 'email': 99,
 'yet': 282,
 'could': 78,
 'let': 151,
 'know': 147,
 'confirmed': 70,
 'corporate': 76,
 'holiday': 128,
 'plan': 197,
 'kindly': 145,
 'arrange': 24,
 'following': 108,
 'accommodation': 1,
 'hotelresort': 132,
 'detail': 92,
 'given': 117,
 'below': 42,
 'serial': 232,
 'number': 183,
 'name': 177,
 'arun': 25,
 'gadamshetty': 111,
 'no': 180,
 'department': 90,
 'spare': 244,
 'manufacturing': 164,
 'contact': 71,
 'place': 196,
 'vivanta': 275,
 'madikeri': 159,
 'coorg': 74,
 'type': 270,
 'superior': 252,
 'charm': 61,
 'check': 63,
 'datetime': 82,
 'dec': 85,
 'pm': 199,
 'comment': 66,
 'requester': 218,
 'honeymoon': 129,
 'trip': 268,
 'traveller': 266,
 'relat

In [192]:
help(TfidfVectorizer)

Help on class TfidfVectorizer in module sklearn.feature_extraction.text:

class TfidfVectorizer(CountVectorizer)
 |  Convert a collection of raw documents to a matrix of TF-IDF features.
 |  
 |  Equivalent to CountVectorizer followed by TfidfTransformer.
 |  
 |  Read more in the :ref:`User Guide <text_feature_extraction>`.
 |  
 |  Parameters
 |  ----------
 |  input : string {'filename', 'file', 'content'}
 |      If 'filename', the sequence passed as an argument to fit is
 |      expected to be a list of filenames that need reading to fetch
 |      the raw content to analyze.
 |  
 |      If 'file', the sequence items must have a 'read' method (file-like
 |      object) that is called to fetch the bytes in memory.
 |  
 |      Otherwise the input is expected to be the sequence strings or
 |      bytes items are expected to be analyzed directly.
 |  
 |  encoding : string, 'utf-8' by default.
 |      If bytes or files are given to analyze, this encoding is used to
 |      decode.
 |

In [193]:
testTfIdf = TfidfVectorizer(vocabulary=vectorizer.vocabulary_)

In [194]:
testTfIdf.fit(textFeaturesTest)
da = testTfIdf.transform(textFeaturesTest)

In [195]:
da.data

array([0.5, 0.5, 0.5, 0.5])

In [196]:
textFeaturesTest

0    hi please cancel booking
dtype: object